The following code needs to be run once to save your credentials to a local cache. You'll need to replace the path to site-packages appropriately...

`run /Users/frank/.virtualenvs/EarthEngine/lib/python2.7/site-packages/ee/authenticate.py`

This was the contents of the old way of authenticating (It still seems to work as of 27 March 2015):

```
#MY_SERVICE_ACCOUNT = '544951530331@developer.gserviceaccount.com'  # replace with your service account
#MY_PRIVATE_KEY_FILE = '/Users/frank/Documents/Src/Git Stuff/EarthEngine/EarthEngineTest-privatekey.p12'       # replace with your private key file path
#ee.Initialize(ee.ServiceAccountCredentials(MY_SERVICE_ACCOUNT, MY_PRIVATE_KEY_FILE))
```


In [1]:
import ee
import time
ee.Initialize()
drive_folder = 'Google EE Exports'

In [2]:
# ROI for clipping
west = -79.0
east = -73.5
south = 42.0
north = 44.0
upstateNyROI = ee.Feature.Rectangle(west, south, east, north)
upstateNyROIString = '[[%f,%f], [%f,%f], [%f,%f], [%f,%f]]'%(west,north,east,north,east,south,west,south)

start_date = '2002-01-01'
end_date = '2014-12-31'

# MODIS LST Images

# Terra
# MODIS/MoD11A1

terra_day_collection = ee.ImageCollection('MODIS/MOD11A1').filterDate(start_date, end_date)\
                   .filterBounds(upstateNyROI).select([0],['Temperature'])
terra_night_collection = ee.ImageCollection('MODIS/MOD11A1').filterDate(start_date, end_date)\
                     .filterBounds(upstateNyROI).select([4],['Temperature'])
terra_collection = ee.ImageCollection(terra_day_collection.merge(terra_night_collection))

# Aqua
# MODIS/MYD11A1

aqua_day_collection = ee.ImageCollection('MODIS/MYD11A1').filterDate(start_date, end_date)\
                   .filterBounds(upstateNyROI).select([0],['Temperature'])
aqua_night_collection = ee.ImageCollection('MODIS/MYD11A1').filterDate(start_date, end_date)\
                     .filterBounds(upstateNyROI).select([4],['Temperature'])
aqua_collection = ee.ImageCollection(aqua_day_collection.merge(aqua_night_collection))

# All of them
collection = ee.ImageCollection(terra_collection.merge(aqua_collection))


quantiles = [i for i in range(0,101,2)]
#print(quantiles)

percentiles = collection.reduce(ee.Reducer.percentile(quantiles)).toFloat().multiply(0.02)

mean = collection.mean().toFloat().multiply(0.02)


#median = collection.median()

In [3]:
task = ee.batch.Export.image(percentiles,'AquaTerraDayNightPercentiles_UpstateNY_2002_2014',
                             {'region':upstateNyROIString,
                              'scale':926,
                              'driveFolder':drive_folder})
task.start()
time.sleep(10)
while task.status()['state'] == 'RUNNING':
  print 'Running...'
  # Perhaps task.cancel() at some point.
  time.sleep(60)
print 'Done.', task.status()

task = ee.batch.Export.image(mean,'AquaTerraDayNightMean_UpstateNY_2002_2014',
                             {'region':upstateNyROIString,
                              'scale':926,
                              'driveFolder':drive_folder})
task.start()
time.sleep(10)
while task.status()['state'] == 'RUNNING':
  print 'Running...'
  # Perhaps task.cancel() at some point.
  time.sleep(60)
print 'Done.', task.status()


Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Done. {u'task_type': u'EXPORT_IMAGE', u'description': u'AquaTerraDayNightPercentiles_UpstateNY_2002_2014', u'output_url': [u'https://drive.google.com/#folders/0B2oo-_fzvmOKRHN2NkJhTnJ6VDg'], u'update_timestamp_ms': 1427673160583, u'creation_timestamp_ms': 1427670214343, u'state': u'COMPLETED', u'start_timestamp_ms': 1427671730452, u'id': u'ZCNYXMOCIRELBPWKUZYDOJDA'}
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Runn

In [4]:
task.status()

{u'creation_timestamp_ms': 1427628863617,
 u'description': u'DayNightPercentiles_UpstateNY',
 u'id': u'NYV2DXDVL5TID5T5OMNK4ZTB',
 u'output_url': [u'https://drive.google.com/#folders/0B2oo-_fzvmOKRHN2NkJhTnJ6VDg'],
 u'start_timestamp_ms': 1427628864942,
 u'state': u'COMPLETED',
 u'task_type': u'EXPORT_IMAGE',
 u'update_timestamp_ms': 1427629137135}

In [5]:
task.cancel()